In [31]:
"""
    Imports
"""

import requests
from io import BytesIO
from PIL import Image
import os
from itertools import islice

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as T
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import BinaryF1Score

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

from utils import *

In [35]:
"""
    https://towardsdatascience.com/contrastive-loss-explaned-159f2d4a87ec for more details 
"""

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        loss_contrastive = torch.mean(label * torch.pow(distance, 2) +
                                      (1 - label) * torch.pow(torch.clamp(self.margin - distance, min=0), 2))

        return loss_contrastive

"""
    https://towardsdatascience.com/a-friendly-introduction-to-siamese-networks-85ab17522942
"""

class SiameseNetworkClassifier(nn.Module):
    def __init__(self, device='mps'):
        super(SiameseNetworkClassifier, self).__init__()

        # Replace with frozen ResNet152 feature layers
        resnet = models.resnet152(pretrained=True)
        # Freeze all layers
        for param in resnet.parameters():
            param.requires_grad = False

        # Split the model into frozen and hot layers
        self.frozen = nn.Sequential(*list(resnet.children())[:-1]) 
        self.hot = nn.Sequential(
            nn.Flatten(),
            nn.Linear(resnet.fc.in_features, 50)
        )

        self.threshold = torch.tensor(0.)

        self.device = torch.device(device)
        self.to(self.device)

    def forward(self, images1, images2):
        output1 = self.hot(self.frozen(images1))
        output2 = self.hot(self.frozen(images2))
        return F.pairwise_distance(output1, output2)
        
    def update_threshold(self, loader, max_batches=None):
        self.eval()
        with torch.no_grad():
            distances = []
            labels = []
            for images1, images2, equals in islice(tqdm(loader, desc='Calculating threshold'), max_batches):
                distance = self.forward(images1.to(self.device), images2.to(self.device))
                distances.append(distance.cpu())
                labels.append(equals)
    
            distances = torch.cat(distances)
            labels = torch.cat(labels)
            log_reg = LogisticRegression(penalty=None)
            log_reg.fit(distances.reshape((-1, 1)), labels)
            self.threshold = (-log_reg.intercept_ / log_reg.coef_).item()

    # TODO refactor this method so we don't have to call .to(self.device) ? 
    def predict(self, images1, images2):
        self.eval()
        with torch.no_grad():
            images1 = images1.to(self.device)
            images2 = images2.to(self.device)
            distances = self.forward(images1, images2)
            return (distances < self.threshold).int().cpu()

def evaluate(model, loader, max_batches=None):
    model.eval()
    with torch.no_grad():
        pos_f1 = BinaryF1Score()
        neg_f1 = BinaryF1Score()
        for images1, images2, label in islice(tqdm(loader, desc='Evaluating model'), max_batches):
            distance = model.forward(images1.to(model.device), images2.to(model.device)).cpu()
            pos_f1.update(distance < model.threshold, label)
            neg_f1.update(distance > model.threshold, 1 - label)
    return (pos_f1.compute() + neg_f1.compute()) / 2
    
def train(model, train_loader, train_threshold_loader, valid_loader, test_loader, epochs=20, lr=1e-4, max_batches=None):
    print("Debug: Initializing ContrastiveLoss and Optimizer")
    criterion = ContrastiveLoss().to(model.device)
    optimizer = torch.optim.Adam(model.hot.parameters(), lr=lr)

    for epoch in range(epochs):
        for images1, images2, label in islice(tqdm(train_loader, desc='Training model'), max_batches):
            model.train()
            images1 = images1.to(model.device)
            images2 = images2.to(model.device)
            label = label.to(model.device)

            optimizer.zero_grad()
            outputs = model.forward(images1, images2)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

        model.update_threshold(train_threshold_loader, max_batches=max_batches)
        
        print(f'Epoch {epoch} | Loss:{loss.item()}')
        print(f'Train fscore: {evaluate(model, train_loader, max_batches=max_batches)}')
        print(f'Valid fscore: {evaluate(model, valid_loader, max_batches=max_batches)}')
    print(f'Test fscore: {evaluate(model, test_loader, max_batches=max_batches)}')

In [36]:
batch_size = 32
max_submit_id = 22661

In [37]:
transform = T.Compose([
    T.Resize(400),
    T.CenterCrop(350),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

df_slices = list(split_dataframe(pd.read_csv('data/train.csv'), (0.9, 0.01, 0.04, 0.05)))

datasets = [ImageDataset(df, transform=transform) for df in df_slices]
loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in datasets]

train_dataset, train_threshold_dataset, valid_dataset, test_dataset = datasets
train_loader, train_threshold_loader, valid_loader, test_loader = loaders

submit_df = pd.read_csv('data/submit.csv')
submit_dataset = ImageDataset(submit_df, transform=transform)
submit_loader = DataLoader(submit_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
model = SiameseNetworkClassifier()
train(model, *loaders, epochs=10, max_batches=300)

/opt/homebrew/Caskroom/miniconda/base/envs/image-comparison/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/image-comparison/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Debug: Initializing ContrastiveLoss and Optimizer


Training model:   0%|          | 0/1864 [00:00<?, ?it/s]

In [30]:
"""
    Check on what's wrong with our model
"""

mislabeled(model, valid_loader)

Button(description='Next Images', style=ButtonStyle())

Output()